In [8]:
! pip -q install google-genai

In [9]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [10]:
from google import genai

client = genai.Client()
modelo = "gemini-2.0-flash"

In [11]:
from IPython.display import HTML, Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [12]:
! pip -q install google-adk

In [13]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings

warnings.filterwarnings("ignore")

In [14]:
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [23]:
def agente_buscador(materia, ano, data_de_hoje):

  buscador = Agent(
      name="agente_buscador",
      model="gemini-2.0-flash",
      instruction="""
        Você é um agente de pesquisa. A sua tarefa é usar a ferramenta de busca do Google (google_search) para pesquisar atividades simples
        e eficientes para o aprendizado de estudantes do primeiro ao quinto ano do Ensino fundamental. Foque em atividades escolares alinhadas
        com a versão mais recente da Base Nacional Comum Curricular do Brasil estabelecida pelo Ministério da Educação. Busque no máximo 10
        atividades escolares relevantes com base na avaliação crítica de educadores sobre elas. Essas atividades devem ter sido publicadas no
        máximo há 1 ano a partir da data de hoje e devem ser adequadas .
        """,
      description="Agente que busca atividades escolares no Google",
      tools=[google_search]
  )

  entrada_do_agente_buscador = f"Matéria: {materia}\nAno: {ano}\nData de hoje: {data_de_hoje}"

  atividades_buscadas = call_agent(buscador, entrada_do_agente_buscador)
  return atividades_buscadas

In [45]:
def agente_planejador(materia, ano, atividades_buscadas):

  planejador = Agent(
      name="agente_planejador",
      model="gemini-2.0-flash",
      instruction="""
        Você é um planejador de conteúdo, especialista em atividades escolares do primeiro ao quinto ano do Ensino Fundamental alinhadas
        com a versão mais recente da Base Nacional Comum Curricular do Brasil estabelecida pelo Ministério da Educação. Com base na lista
        de atividades buscadas Você deve: usar a ferramenta de busca do Google (google_search) para criar questões que serão utilizadas em
        atividades escolares adequadas para a matéria e para o ano do Ensino Fundamental que foi informado. Use a criatividade para mesclar
        tipos diferentes de questões, de forma a promover o aprendizado de forma lúdica e variada. Ao final, você irá escolher e retornar
        cinco questões que melhor atendam aos parâmetros estabelecidos.
        """,
      description="Agente que planeja questões para atividades escolares",
      tools=[google_search]
  )

  entrada_do_agente_buscador = f"Matéria: {materia}\nAno: {ano}\nData de hoje: {atividades_buscadas}"

  questoes_elaboradas = call_agent(planejador, entrada_do_agente_buscador)
  return questoes_elaboradas

In [46]:
def agente_redator(materia, ano, questoes_elaboradas):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
          Você é um redator criativo especializado em criar atividades escolares para estudantes do primeiro ao quinto ano do Ensino Fundamental
          que estejam alinhadas com a Base Nacional Comum Curricular do Brasil estabelecida pelo Ministério da Educação. Você é capaz de elaborar
          atividades escolares que contenham de 3 a 5 questões de forma que possam ser inseridas em uma única folha de papel tamanho A4 incluindo
          o cabeçalho onde deve sempre constar as seguintes informações: Aluno; Professor; Data. O cabeçalho deve conter os espaços apropriados
          para a escrita à mão e por extenso do nome do aluno, do nome do professor e da data. Utilize as questões elaboradas para fazer o layout
          da atividade escolar atendendo aos parâmetros estabelecidos.
          """,
        description="Agente que cria o layout da atividade"
    )
    entrada_do_agente_redator = f"Matéria: {materia}\nAno: {ano}\nQuestões Elaboradas: {questoes_elaboradas}"

    rascunho_gerado = call_agent(redator, entrada_do_agente_redator)
    return rascunho_gerado

In [47]:
def agente_revisor(materia, ano, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um editor e revisor de atividades escolares do primeiro ao quinto ano do Ensino Fundamental que estejam de acordo com as
            diretrizes da Base Nacional Comum Curricular definida pelo Ministério da Educação. Revise o rascunho da atividade escolar que foi
            elaborada, verificando a clareza dos enunciados, concisão das orientações e as normas gramaticais da língua portuguêsa do Brasil
            com relação à forma correta da escrita. Se o rascunho estiver bom, responda apenas 'A atividade está ótima e pronta para ser
            utilizada!'. Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Matéria: {materia}\nAno: {ano}\nLayout: {rascunho_gerado}"
    rascunho_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return rascunho_revisado

In [51]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("Bem-vindo(a) ao Criador de Atividades Escolares do 1º ao 5º ano do Ensino Fundamental!")

materia = input("Qual a matéria escolar? ")
if not materia:
  print("Por favor, insira o nome da matéria escolar.")
else:
  ano = input('Qual o ano escolar? ')
  if not ano:
    print("Por favor, insira o ano escolar.")
  else:
    print(f"Excelente! Vamos criar uma atividade de {materia} para o {ano} ano.")

  atividade = agente_buscador(materia, ano, data_de_hoje)
  print("\nResultado do Agente Buscador:\n")
  display(to_markdown(atividade))

  questoes = agente_planejador(materia, ano, atividade)
  print("\nResultado do Agente Planejador:\n")
  display(to_markdown(questoes))

  rascunho = agente_redator(materia, ano, questoes)
  print("\nResultado do Agente Redator:\n")
  display(to_markdown(rascunho))

  atividade_final = agente_revisor(materia, ano, rascunho)
  print("\nResultado do Agente Revisor:\n")
  display(to_markdown(atividade_final))

Bem-vindo(a) ao Criador de Atividades Escolares do 1º ao 5º ano do Ensino Fundamental!
Qual a matéria escolar? ciências
Qual o ano escolar? 3º ano
Excelente! Vamos criar uma atividade de ciências para o 3º ano ano.

Resultado do Agente Redator:



> ## Atividade de Ciências - 3º Ano
> 
> **Aluno:**____________________________________________________________
> 
> **Professor:**__________________________________________________________
> 
> **Data:** ____/____/____
> 
> 1.  **Ciclo da Água:** Desenhe e explique as etapas do ciclo da água (evaporação, condensação, precipitação e infiltração). Em seguida, discuta em grupo como a poluição pode afetar esse ciclo e quais ações podemos tomar para protegê-lo.
> 
> 2.  **Seres Vivos e seus Habitats:** Crie um mapa mental que mostre diferentes tipos de seres vivos (animais, plantas, etc.) e seus habitats. Inclua exemplos de adaptações que permitem a cada ser vivo sobreviver em seu ambiente.
> 
> 3.  **Materiais e suas Características:** Realize um experimento simples para comparar as características de diferentes materiais (madeira, plástico, metal, papel). Anote suas observações sobre a textura, resistência e flexibilidade de cada um e discuta como essas características influenciam o uso desses materiais em objetos do dia a dia.
> 
> 4.  **Classificação dos Animais:** Elabore um jogo de cartas com diferentes animais, classificando-os de acordo com suas características (vertebrados/invertebrados, alimentação, habitat). Utilize as cartas para jogar um jogo da memória ou um jogo de adivinhação, incentivando o aprendizado de forma lúdica.
> 
> 5.  **Alimentos e Nutrição:** Monte uma pirâmide alimentar com recortes de revistas e embalagens de alimentos. Explique a importância de cada grupo alimentar para a nossa saúde e crie um cardápio equilibrado para um dia, incluindo opções saudáveis para o café da manhã, almoço, jantar e lanches.
> 



Resultado do Agente Revisor:



> A atividade está ótima e pronta para ser utilizada!
